<a href="https://colab.research.google.com/github/MarcosRolando/OrgaDeDatosTP2/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xgboost
import xgboost as xgb
import pandas as pd
import numpy as np
import math as mt
import regex as re

In [2]:
def preprocess_dataframe(df):

  df.fillna(value=0, inplace=True) #Reemplazamos NAN por 0, ya que NAN rompe a Tensorflow

  #Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
  #Estos particularmente son whitespace, coma y parentesis por ejemplo.
  df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                    'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                    'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

  df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
  df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) #0 corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won. Asi tenemos un problema de clasificacion binario que puede entender la red neuronal.

  df.loc[:, 'Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], 'coerce',
                                                                  format='%m/%d/%Y')
  df.loc[:, 'Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df.loc[:, 'Last_Modified_Date'] = pd.to_datetime(df['Last_Modified_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df.loc[:, 'Quote_Expiry_Date'] = pd.to_datetime(df['Quote_Expiry_Date'], 'coerce', format='%m/%d/%Y')

  df.loc[:, 'Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df.loc[:, 'Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], 'coerce', format='%m/%d/%Y')

  df = df[df['Opportunity_ID'] != 9773] #Hardcodeo este filtrado porque el id 9773 tiene mal cargada la fecha de delivery end, dando una diferencia de 200 anios xd"

  #Pongo .loc porque pandas me jode con warnings que son falsos positivos de slice copy"
  #Gracias Pandas!"

  #Creamos una nueva columna (Feature Engineering) que contiene la longitud en dias 
  #estimada de la operacion. En el informe habiamos encontrado que aparentaba haber
  #una relacion cuadratica de decrecimiento a medida que aumentaban los dias donde disminuia
  #la chance de completar la operacion.
  df['Planned_Opportunity_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']
  df.loc[:, 'Planned_Opportunity_Time'] = df['Planned_Opportunity_Time'].dt.days
  df['Planned_Opportunity_Time'] = df['Planned_Opportunity_Time'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
  df['Planned_Opportunity_Time'] = df.groupby('Opportunity_ID')['Planned_Opportunity_Time'].transform('max')

  df['End_Date_vs_Modified_Date_Difference'] = df['Planned_Delivery_End_Date'] - df['Last_Modified_Date']
  df.loc[:, 'End_Date_vs_Modified_Date_Difference'] = df['End_Date_vs_Modified_Date_Difference'].dt.days
  df['End_Date_vs_Modified_Date_Difference'] = df['End_Date_vs_Modified_Date_Difference'].replace({np.nan:10.0})
  df['End_Date_vs_Modified_Date_Difference'] = df.groupby('Opportunity_ID')['End_Date_vs_Modified_Date_Difference'].transform('max')

  df['Expiry_Date_vs_Modified_Date_Difference'] = df['Planned_Delivery_End_Date'] - df['Quote_Expiry_Date']
  df.loc[:, 'Expiry_Date_vs_Modified_Date_Difference'] = df['Expiry_Date_vs_Modified_Date_Difference'].dt.days
  df['Expiry_Date_vs_Modified_Date_Difference'] = df['Expiry_Date_vs_Modified_Date_Difference'].replace({np.nan:10.0})
  df['Expiry_Date_vs_Modified_Date_Difference'] = df.groupby('Opportunity_ID')['Expiry_Date_vs_Modified_Date_Difference'].transform('max')

  df['Acc_vs_Opp_Creation_Dates'] = df['Opportunity_Created_Date'] - df['Account_Created_Date']
  df.loc[:, 'Acc_vs_Opp_Creation_Dates'] = df['Acc_vs_Opp_Creation_Dates'].dt.days
  df['Expiry_Date_vs_Modified_Date_Difference'] = df['Acc_vs_Opp_Creation_Dates'].replace({np.nan:10})
  df['Acc_vs_Opp_Creation_Dates'] = df.groupby('Opportunity_ID')['Acc_vs_Opp_Creation_Dates'].transform('max')

  #Agrego una columna que indique la cantidad de productos que tiene esa
  #oportunidad
  df['Product'] = 1
  df['Product_Amount'] = df.groupby('Opportunity_ID')['Product'].transform(lambda x: x.sum())


  #Pasamos todo a dolares
  currency_conversion = {'AUD':0.707612, 'EUR':1.131064, 'GBP':1.318055, 'JPY':0.008987, 'USD':1.0}
  df['Total_Taxable_Amount_Currency'] = df[['Total_Taxable_Amount_Currency']].replace(currency_conversion)
  df['Total_Taxable_Amount'] = df['Total_Taxable_Amount_Currency'] * df['Total_Taxable_Amount']
  #df['Total_Taxable_Amount'] = df.groupby("Opportunity_ID")['Total_Taxable_Amount'].transform("sum")
  df['Total_Taxable_Amount'] = df['Total_Taxable_Amount'] * df['Product_Amount']

  #Modifico la columna Brand para que en vez de decir que marca es, solo diga
  #si tiene o no marca. Es importante aclarar que verificamos que siempre que una oportunidad
  #tiene un producto con marca entonces todos sus productos tienen marca. Esto se cumple
  #tanto en el set de entrenamiento como en el de test, por lo tanto al hacer drop_duplicates
  #no nos va a pasar nunca el caso donde nos pudieramos quedar con una entrada de producto
  #sin marca mientras que algun otro producto si tuviera, ya que confirmamos que o todos tienen
  #marca o ninguno tiene.
  df.loc[df['Brand'] == 'None', 'Brand'] = 'No'
  df.loc[df['Brand'] != 'No', 'Brand'] = 'Yes'

  #Agrego una columna que indica si el owner de la cuenta es el mismo que el de la oportunidad
  #o no
  df['Same_Owner'] = (df['Account_Owner'] == df['Opportunity_Owner'])
  df['Same_Owner'] = df['Same_Owner'].replace({False:'No', True:'Yes'})

  #Agrego columna que indica si el ultimo que modifico la oportunidad es el mismo que el opportunity owner
  df['Same_Owner_Modifier'] = (df['Last_Modified_By'] == df['Opportunity_Owner'])
  df['Same_Owner_Modifier'] = df['Same_Owner_Modifier'].replace({False:'No', True:'Yes'})

  #Agrego una columna que indica si tiene o no fecha de expiracion
  df['Quote_Expiry_Date'] = (df['Quote_Expiry_Date'] != 'NaT')
  df.rename(columns={'Quote_Expiry_Date':'Has_Expiry_Date'}, inplace=True)
  df['Has_Expiry_Date'] = df['Has_Expiry_Date'].replace({True:'Yes',False:'No'})

  #Reemplazo las 4 columnas de aprobacion por solo 2 columnas que indiquen si tuvo la aprobacion
  #de delivery y burocratica o no. Recalco que si nunca la necesito seria equivalente a si
  #la necesito y la consiguio.
  df['Delivery_Approved'] = df['Pricing_Delivery_Terms_Quote_Appr'] + df['Pricing_Delivery_Terms_Approved']
  df['Delivery_Approved'] = df['Delivery_Approved'].replace({0:1, 1:0, 2:1})
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_0_Approval'] + df['Bureaucratic_Code_0_Approved']
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_Approved'].replace({0:1, 1:0, 2:1})
  df['Approved'] = df['Delivery_Approved'] & df['Bureaucratic_Code_Approved']

  #Cambio TRF por una columna que es el valor maximo de los TRF de la oportunidad
  df["TRF"] = df.groupby("Opportunity_ID")["TRF"].transform("max")

  def combineProducts(x):
    products = ""
    added = []
    for product in x:
      product = re.findall('\d+', product)[0]
      if added.count(product) == 0:
        products += product
        added.append(product)
    return products

  #Junto todos los productos en una sola entrada
  df['Products'] = df.groupby('Opportunity_ID')['Product_Family'].transform(combineProducts)

  #Pruebo volar duplicados, solo cambia el producto. Si el producto no importa
  #entonces volar duplicados no deberia importar. Obviamente vuelo el producto en el que
  #quede tambien.
  df.drop_duplicates('Opportunity_Name',inplace=True)
  df.drop(columns=['Product_Name','Product_Family','Opportunity_Name'],inplace=True)

  #Normalizo las columnas numericas
  normalized_columns = ['ASP_converted','TRF','Total_Taxable_Amount', 'Product_Amount',
                        'Planned_Opportunity_Time', 'End_Date_vs_Modified_Date_Difference',
                        'Expiry_Date_vs_Modified_Date_Difference', 'Acc_vs_Opp_Creation_Dates']

  for column in normalized_columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

  #Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
  #Algunas columnas borradas son porque pienso que no tienen incidencia, ir viendo.
  #TODO: Analizar si el Sales_Contract_No no es que importe el numero en si, sino si tiene
  #o no tiene numero de contrato. Por ahora no lo meto como input.
  #TODO: Ver el mismo tema con la columna 'Price', la mayoria tiene None u Other
  #y solo unos pocos tienen precio numerico. Quiza importe que tenga precio o no tenga,
  #o si no tiene precio quiza importe si es None u Other. Por ahora no lo pongo
  #como input.
  df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                  'Prod_Category_A', 'ID', 'Opportunity_ID', 
                   'Actual_Delivery_Date'],inplace=True)

  #Drop columnas que quiza podamos usar pero por ahora no las uso
  df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                  'Last_Modified_Date',
                  'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                  'Month',
                  'Delivery_Year',
                  'Price','ASP','Total_Amount_Currency',
                  'Total_Amount','Total_Taxable_Amount_Currency', 'Currency',
                   'Product_Category_B','Last_Modified_By', 'Account_Owner',
                   'Opportunity_Owner','Account_Name','Product_Type',
                   'Billing_Country', 'Sales_Contract_No',
                   'Product', 'Products','Territory']
                   ,inplace=True)


  df = df[['Source', 'Opportunity_Type', 'Brand', 'Planned_Opportunity_Time','Total_Taxable_Amount',
           'Product_Amount','ASP_converted','TRF', 'Bureaucratic_Code','Quote_Type',
           'Expiry_Date_vs_Modified_Date_Difference', "Stage"]]

  #Agrego esto para que se pueda usar XGBoost
  onehot_columns = ['Source', 'Opportunity_Type', 'Brand', 'Bureaucratic_Code','Quote_Type']

  onehot_df = df[onehot_columns]
  onehot_df = pd.get_dummies(onehot_df, columns = onehot_columns)
  df_no_one_hot = df.drop(onehot_columns, axis = 1)
  df_one_hot = pd.concat([df_no_one_hot, onehot_df], axis = 1)

  #Definimos que tipo de feature es cada columna

  #Debemos separar algunos de los registros para armar un set de test propio (no el de la catedra). De esta forma sabremos rapidamente
  #si nuestro modelo esta dando resultados optimos o no sin necesidad de estar subiendo el TP a Kaggle constantemente.
  #Sin embargo, no queremos usar tantos registros ya que estariamos disminuyendo el set de entrenamiento considerablemente.
  #Podemos empezar reservando 2000 registros para el test de prueba y ver que onda. Pasariamos de tener 16 mil a 14 mil 
  #registros para el set de entrenamiento, no es una perdida importantisima creo en principio, asi que arrancamos con eso.

  #Por otro lado, nuestro test de prueba deberia tener un 50 50 de Closed Won y Closed Lost, por lo que no podemos elegir asi nomas
  #al azar.

  return df_one_hot

In [3]:
def log_loss(predictions, results):
  acum = 0
  if len(predictions) != len(results):
    print("Predicciones y resultados de distinto tamaño")
    return None
  for i in range(len(predictions)):
    if results[i] == 1:
      acum += mt.log(predictions[i])
    elif results[i] == 0:
      acum += mt.log(1-predictions[i])
    else:
      print("Resultado inesperado")
      return None
  return -acum/len(predictions)

In [4]:
# read in data
#dtrain = xgb.DMatrix("/content/Train_TP2_Datos_2020-2C.csv?format=csv")
#dtest = xgb.DMatrix("/content/Test_TP2_Datos_2020-2C.csv?format=csv")
df_train = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')
df_train = preprocess_dataframe(df_train)
columns_order = df_train.columns
test_lines = 200
np.random.seed(931)
drop_indices = np.random.choice(df_train.index, test_lines, replace=False)
df_test = df_train.loc[drop_indices, :]
df_train.drop(drop_indices, inplace=True)

x_train = df_train.drop("Stage", axis = 1)
y_train = df_train["Stage"]
x_test = df_test.drop("Stage", axis = 1)
y_test = df_test["Stage"]

"""x_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
x_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]"""


# specify parameters via map
#param = {"max_depth":4, "eta":0.15, "subsample":0.3, "grow_policy":"lossguide", "objective":"binary:logistic" }
"""param = {"max_depth":4, "eta":0.15, "subsample":0.3, "colsample_bylevel":0.5, "colsample_bytree":1, 
         "min_child_weight":5, "objective":"binary:logistic" }"""
param = {"max_depth":4, "eta":0.15, "subsample":0.6, "colsample_bylevel":0.75, 
         "colsample_bytree":0.5, "objective":"binary:logistic" }
epochs = 500
#epochs = 60
#bst = xgb.train(param, xgb.DMatrix(data=x_train,label=y_train), epochs)
"""bst = xgb.train(param, xgb.DMatrix(data=x_train,label=y_train), epochs, 
                early_stopping_rounds=100,
                evals= [ (xgb.DMatrix(data=x_train,label=y_train),'train'),(xgb.DMatrix(data=x_test,label=y_test),'eval')])"""
bst = xgb.train(param, xgb.DMatrix(data=x_train,label=y_train), epochs, 
                early_stopping_rounds=20,
                evals= [(xgb.DMatrix(data=x_test,label=y_test),'eval')])
# make prediction
preds = bst.predict(xgb.DMatrix(data=x_test,label=y_test))

print("Loss entrenamiento: " + str(log_loss(bst.predict(xgb.DMatrix(data=x_train,label=y_train)), y_train.to_list())))
print("Loss test: " + str(log_loss(preds, y_test.to_list())))

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


[0]	eval-error:0.25
Will train until eval-error hasn't improved in 20 rounds.
[1]	eval-error:0.23
[2]	eval-error:0.21
[3]	eval-error:0.195
[4]	eval-error:0.205
[5]	eval-error:0.185
[6]	eval-error:0.195
[7]	eval-error:0.185
[8]	eval-error:0.19
[9]	eval-error:0.195
[10]	eval-error:0.195
[11]	eval-error:0.2
[12]	eval-error:0.2
[13]	eval-error:0.195
[14]	eval-error:0.195
[15]	eval-error:0.195
[16]	eval-error:0.195
[17]	eval-error:0.2
[18]	eval-error:0.205
[19]	eval-error:0.195
[20]	eval-error:0.19
[21]	eval-error:0.2
[22]	eval-error:0.19
[23]	eval-error:0.19
[24]	eval-error:0.195
[25]	eval-error:0.2
Stopping. Best iteration:
[5]	eval-error:0.185

Loss entrenamiento: 0.432451281043769
Loss test: 0.4445458336318652


In [5]:

frio_test_df = pd.read_csv('/content/Test_TP2_Datos_2020-2C.csv')
frio_test_df['Stage'] = 'Closed Won' #Esto esta solo para que funque todo, no lo uso. No se bien como armarlo sin los labels de Stage. TODO: Averiguar como es!
aux_df = frio_test_df[['Opportunity_ID']] #Esta columna la vuela el preprocesado sino
frio_test_df = preprocess_dataframe(frio_test_df)

for col in columns_order:
  if not col in frio_test_df.columns:
    frio_test_df[col] = 0

frio_test_df = frio_test_df[columns_order]
x_real_test = frio_test_df.drop("Stage", axis = 1)
y_real_test = frio_test_df["Stage"]

m = xgb.DMatrix(data = x_real_test, label = y_real_test)
predictions = bst.predict(m)

aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True) #Lo hacia el preprocesado pero es verdad que lo copie antes a este xd, perdon Agus, paja de dejarlo lindo.
aux_df['Target'] = predictions

#aux_df['Target'] = aux_df.groupby(by='Opportunity_ID').transform(lambda x: x.mean())
#aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True)

aux_df.to_csv('prediccionesFrioFrio.csv', index=False)
'''
df = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')
df = df[(df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost')]
df = df[df['Opportunity_ID'] != 9773]
df = df[['Opportunity_ID']]
df.drop_duplicates(subset='Opportunity_ID', inplace=True)
np.random.seed(1)
drop_indices = np.random.choice(df.index, test_lines, replace=False)
df.drop(drop_indices, inplace=True)
predictions = model.predict(ds)
df['Target'] = predictions
df.to_csv('prediccionesFrioFrio.csv', index=False)
'''

"\ndf = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')\ndf = df[(df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost')]\ndf = df[df['Opportunity_ID'] != 9773]\ndf = df[['Opportunity_ID']]\ndf.drop_duplicates(subset='Opportunity_ID', inplace=True)\nnp.random.seed(1)\ndrop_indices = np.random.choice(df.index, test_lines, replace=False)\ndf.drop(drop_indices, inplace=True)\npredictions = model.predict(ds)\ndf['Target'] = predictions\ndf.to_csv('prediccionesFrioFrio.csv', index=False)\n"